<a href="https://colab.research.google.com/github/fedhere/PUS2020_KLee/blob/master/HW4/reviewed_PUS2020_HW_4_how_to_read_and_map_census_data_in_python_K.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  you should change this -> this is your homework author: Federica Bianco (@fedhere)

Notebook with examples of how to read and plot cenusus data in python



In [ ]:
!pip install census
!pip install CensusData

     |████████████████████████████████| 23.2MB 179kB/s 
  Created wheel for CensusData: filename=CensusData-1.9-cp36-none-any.whl size=24706266 sha256=763d07ff804983cd0064b971db75af955945fe1bda56d0a248930cb1407be45e
  Stored in directory: /root/.cache/pip/wheels/39/4c/19/0770e56b66921fd366a5d32382b3db71e54aa98259f6bc88cb
Successfully built CensusData


In [ ]:
!pip install us


     |████████████████████████████████| 133kB 3.9MB/s 


In [ ]:
!pip install geopandas
!pip install descartes

In [ ]:
!pip install shapely

In [ ]:
# module for mapping; if you are familiar with pandas this should be intuitive
import geopandas as gpd

In [ ]:
import pandas as pd
import numpy as np
import pylab as plt

%pylab inline

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

# mounting your drive (My Drive) to read the api key from a csv file. Look at the readme for more info

In [ ]:
cd /content/gdrive/My\ Drive

In [ ]:
pwd

In [ ]:
ls

## read about the census API DO NOT PRINT IT! your API key should be secret

In [ ]:
myAPIkey =  pd.read_csv("myapi.csv")["census"]

# example of reading race data at the county level.

In [ ]:
# import modules to get census data
# register with your API key
import censusdata
from census import Census
from us import states
c = Census(myAPIkey)

### look in the census variables the ones apprioriate for race. 

We are going to use the 5 year American Community Survey (```acs5```), which is ran by the Census Boureaum, but it is not the Census, in that it does not aspire to collect data about every US resident, but it is run more often (5 years). There is also an annual American Community Survey (```acs1```)

In [ ]:
censusdata.search('acs5', 2018, 'concept', 'race')

These are the variables that interest us. They start with B02001

('B02001_001E', 'RACE', 'Estimate!!Total'),

('B02001_002E', 'RACE', 'Estimate!!Total!!White alone'),

 ('B02001_003E', 'RACE', 'Estimate!!Total!!Black or African American alone'),

 

In [ ]:
censusdata.printtable(censusdata.censustable('acs5', 2018, 'B02001')), 

get the state code for DE

In [ ]:
censusdata.geographies(censusdata.censusgeo([('state', '*')]), 'acs5', 2018)

get the data at county granularity. Tract granularity is not available for these data in this survey

In [ ]:
total18 = censusdata.download('acs5', 2018, 
                             censusdata.censusgeo([('state', '10'), 
                                                   ('county', '*')]),
                             ["B02001_001E"])
wDEcounty18 = censusdata.download('acs5', 2018,
                             censusdata.censusgeo([('state', '10'), 
                                                   ('county', '*')]),
                             ["B02001_002E"])
bDEcounty18 = censusdata.download('acs5', 2018,
                             censusdata.censusgeo([('state', '10'), 
                                                   ('county', '*')]),
                             ["B02001_003E"])
bDEcounty18

# get the same data for the _previous_ ACS5 as well (remember, the ACS5 is done every 5 years)

In [ ]:
censusdata.search('acs5', 2013, 'concept', 'race')

In [ ]:
censusdata.printtable(censusdata.censustable('acs5', 2013, 'B02001')), 

In [ ]:
censusdata.geographies(censusdata.censusgeo([('state', '*')]), 'acs5', 2013)

In [ ]:
total13 = censusdata.download('acs5', 2013,
                             censusdata.censusgeo([('state', '10'), 
                                                   ('county', '*')]),
                             ["B02001_001E"])
wDEcounty13 = censusdata.download('acs5', 2013,
                             censusdata.censusgeo([('state', '10'), 
                                                   ('county', '*')]),
                             ["B02001_002E"])
bDEcounty13 = censusdata.download('acs5', 2013,
                             censusdata.censusgeo([('state', '10'), 
                                                   ('county', '*')]),
                             ["B02001_003E"])
bDEcounty13

## merge the blackpop dataframe with the total dataframe to extract the fraction of black people. 
## merge the whitepop dataframe with the total dataframe to extract the fraction of white people. 
 

In [ ]:
bDEcounty18 = bDEcounty18.merge(total18, left_index=True, right_index=True)
bDEcounty13 = bDEcounty13.merge(total13, left_index=True, right_index=True)

wDEcounty18 = wDEcounty18.merge(total18, left_index=True, right_index=True)
wDEcounty13 = wDEcounty13.merge(total13, left_index=True, right_index=True)

In [ ]:
#bDEcounty18 = bDEcounty18.merge(total18, left_on=bDEcounty18.index, right_on=total18.index)
#bDEcounty13 = bDEcounty13.merge(total13, left_on=bDEcounty13.index, right_on=total13.index)

#wDEcounty18 = wDEcounty18.merge(total18, left_on=wDEcounty18.index, right_on=total18.index)
#wDEcounty13 = wDEcounty13.merge(total13, left_on=wDEcounty13.index, right_on=total13.index)

In [ ]:
bDEcounty18["black_fraction"] =  bDEcounty18["B02001_003E"] / total18["B02001_001E"] 
bDEcounty13["black_fraction"] =  bDEcounty13["B02001_003E"] / total13["B02001_001E"] 

wDEcounty18["white_fraction"] =  wDEcounty18["B02001_002E"] / total18["B02001_001E"] 
wDEcounty13["white_fraction"] =  wDEcounty13["B02001_002E"] / total13["B02001_001E"] 

## the county index needs to be manipulated
The county name in the dataframes you created needs to be extracted from the index. 
Use the method ```split``` of a string on each row of the index. you can write a function such as

```
def getcounty(cname):
  return cname.split(':')[-1]
```
a function like this will extract the county from the name, this function can be passed to a column (the index in this case) of a dataframe to create a new colum as a modification of the previous one.

In [ ]:
def scounty(cname):
  return cname.split(':')[-1]

In [ ]:
#bDEcounty18["key_0"].astype(str).values[0]
#scounty(wDEcounty18['key_0'].astype(str).values[0])

In [ ]:
#wDEcounty18["county"] = wDEcounty18["key_0"].astype(str).map(scounty)
#bDEcounty18["county"] = bDEcounty18["key_0"].astype(str).map(scounty)

#wDEcounty13["county"] = wDEcounty13["key_0"].astype(str).map(scounty)
#bDEcounty13["county"] = bDEcounty13["key_0"].astype(str).map(scounty)

In [ ]:
# manipulate index to extract the county
wDEcounty18["county"] = wDEcounty18.index.astype(str).map(scounty)
bDEcounty18["county"] = bDEcounty18.index.astype(str).map(scounty)

wDEcounty13["county"] = wDEcounty13.index.astype(str).map(scounty)
bDEcounty13["county"] = bDEcounty13.index.astype(str).map(scounty)

# FBB you should print out the stages of your data processing so that the reader can follow what is going on. Especially if you are working in a notebook (if you are working on a python script, especially if it is intended as part of a pipeline, of course, you do not want to print out unnecessary info, but notebooks are for display)

# Get the DE shape file. 

Lets get it at the census tract level, even if we only have data at the county level right now
## download census tracts file from here  https://www2.census.gov/geo/tiger/TIGER2018/TRACT/tl_2018_10_tract.zip
## and here  https://www2.census.gov/geo/tiger/TIGER2018/COUNTY/tl_2018_us_county.zip


In [ ]:
!wget https://www2.census.gov/geo/tiger/TIGER2018/TRACT/tl_2018_10_tract.zip

In [ ]:
#save it on the drive, unzip it
!unzip tl_2018_10_tract.zip

In [ ]:
gde = gpd.GeoDataFrame.from_file("tl_2018_10_tract.shp")
gde.shape

In [ ]:
gde

In [ ]:
gde.plot(color="None", edgecolor='k',figsize=(5,10))

This map includes census tracts that are all water in the Delaware rever. Remove them using the ALAND column, which is the area of the tract that is not water

In [ ]:
#delete
# only select where the land is >0
gde = gde[gde["ALAND"] > 0]
gde.head()

Now merge the geo data frame with the data frame. 
IMPORTANT! start the merge from the geodataframe:
```gde.merge...```. 
if you start it from the dataframe (```wDEcounty.merge```) you end up with a dataframe, not a geodataframe

In [ ]:
whiteDE18 = gde.merge(wDEcounty18, left_on= "COUNTYFP", right_on="county")
whiteDE13 = gde.merge(wDEcounty13, left_on= "COUNTYFP", right_on="county")

blackDE18 = gde.merge(bDEcounty18, left_on= "COUNTYFP", right_on="county")
blackDE13 = gde.merge(bDEcounty13, left_on= "COUNTYFP", right_on="county")

In [ ]:
blackDE18

In [ ]:
# map the fraction of Black populatoin in 2018 and 2013
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(121)
blackDE13.plot(column="black_fraction", #scheme='Quantiles',
            ax=ax, legend=True, vmin=0.05, vmax=0.30)
ax.axis('off');
plt.title("Black fraction 2013")

ax = fig.add_subplot(122)
blackDE18.plot(column="black_fraction", 
               ax=ax, legend=True, vmin=0.05, vmax=0.30);
ax.axis('off');
plt.title("Black fraction 2018");

In [ ]:
# do the same for the white population
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(121)
whiteDE13.plot(column="white_fraction", #scheme='Quantiles',
            ax=ax, legend=True, vmin=0.65, vmax=0.90)
ax.axis('off');
plt.title("White fraction 2013")

ax = fig.add_subplot(122)
whiteDE18.plot(column="white_fraction", 
               ax=ax, legend=True, vmin=0.75, vmax=0.90);
ax.axis('off');
plt.title("White fraction 2018");

# FBB all figures in this class need captions. Normally I do not give any credit for work that has figures that are not described. I will fige partial credits this time since all students did not put captions

# extra credit

Can you get a map of a demographic variable (e.g. age, education...) at the tract level?

In [ ]:
censusdata.search('acs5', 2018, 'concept', 'age')

'B01001_001E', 'SEX BY AGE', 'Estimate!!Total'

In [ ]:
censusdata.printtable(censusdata.censustable('acs5', 2018, 'B01001')), 

B01001_001E  | SEX BY AGE                     | !! Estimate Total  

B01001_002E  | SEX BY AGE                     | !! !! Estimate Total Male

B01001_026E  | SEX BY AGE                     | !! !! Estimate Total Female

B01001_011E  | SEX BY AGE                     | !! !! !! Estimate Total Male 25 to 29 years

B01001_035E  | SEX BY AGE                     | !! !! !! Estimate Total Female 25 to 29 years


In [ ]:
censusdata.geographies(censusdata.censusgeo([('state', '10'), ('county', '*')]), 'acs5', 2018)

In [ ]:
censusdata.geographies(censusdata.censusgeo([('state', '10'), ('county', '003'), ('tract', '*')]), 'acs5', 2018) 

In [ ]:
age_total18 = censusdata.download('acs5', 2018, censusdata.censusgeo([('state', '10'), ('county', '*'), ('tract', '*')]), ["B01001_001E"])

# by sex
mDEtract18 = censusdata.download('acs5', 2018, censusdata.censusgeo([('state', '10'), ('county', '*'), ('tract', '*')]), ["B01001_002E"])
fDEtract18 = censusdata.download('acs5', 2018, censusdata.censusgeo([('state', '10'), ('county', '*'), ('tract', '*')]), ["B01001_026E"])

# age between 25 and 29
mDEtract18_2529 = censusdata.download('acs5', 2018, censusdata.censusgeo([('state', '10'), ('county', '*'), ('tract', '*')]), ["B01001_011E"])
fDEtract18_2529 = censusdata.download('acs5', 2018, censusdata.censusgeo([('state', '10'), ('county', '*'), ('tract', '*')]), ["B01001_035E"])

In [ ]:
mDEtract18 = mDEtract18.merge(age_total18, left_index=True, right_index=True)
fDEtract18 = fDEtract18.merge(age_total18, left_index=True, right_index=True)

mDEtract18_2529 = mDEtract18_2529.merge(age_total18, left_index=True, right_index=True)
fDEtract18_2529 = fDEtract18_2529.merge(age_total18, left_index=True, right_index=True)

In [ ]:
mDEtract18["male_fraction"] =  mDEtract18["B01001_002E"] / age_total18["B01001_001E"] 
fDEtract18["female_fraction"] =  fDEtract18["B01001_026E"] / age_total18["B01001_001E"] 

mDEtract18_2529["male2529_fraction"] =  mDEtract18_2529["B01001_011E"] / age_total18["B01001_001E"] 
fDEtract18_2529["female2529_fraction"] =  fDEtract18_2529["B01001_035E"] / age_total18["B01001_001E"] 

In [ ]:
def scounty(cname):
  return cname.split(':')[-1]

In [ ]:
mDEtract18["tract"] = mDEtract18.index.astype(str).map(scounty)
fDEtract18["tract"] = fDEtract18.index.astype(str).map(scounty)

mDEtract18_2529["tract"] = mDEtract18_2529.index.astype(str).map(scounty)
fDEtract18_2529["tract"] = fDEtract18_2529.index.astype(str).map(scounty)

In [ ]:
gde.plot(color="None", edgecolor='k')

In [ ]:
gde

In [ ]:
fDEtract18_2529

In [ ]:
maleDE18 = gde.merge(mDEtract18, left_on= "TRACTCE", right_on="tract")
femaleDE18 = gde.merge(fDEtract18, left_on= "TRACTCE", right_on="tract")

maleDE18_2529 = gde.merge(mDEtract18_2529, left_on= "TRACTCE", right_on="tract")
femaleDE18_2529 = gde.merge(fDEtract18_2529, left_on= "TRACTCE", right_on="tract")

In [ ]:
print(maleDE18["male_fraction"].max())
print(maleDE18["male_fraction"].min())
print("")
print(femaleDE18["female_fraction"].max())
print(femaleDE18["female_fraction"].min())
print("")
print(maleDE18_2529["male2529_fraction"].max())
print(maleDE18_2529["male2529_fraction"].min())
print("")
print(femaleDE18_2529["female2529_fraction"].max())
print(femaleDE18_2529["female2529_fraction"].min())

In [ ]:
b# Total Male & Female
fig = plt.figure(figsize=(15,10))
ax = fig.add_subplot(121)
maleDE18.plot(column="male_fraction", #scheme='Quantiles',
            ax=ax, legend=True, vmin=0.3, vmax=0.85)
ax.axis('off');
plt.title("Male fraction 2018")

ax = fig.add_subplot(122)
femaleDE18.plot(column="female_fraction", 
               ax=ax, legend=True, vmin=0.15, vmax=0.70);
ax.axis('off');
plt.title("Female fraction 2018");

# FBB this is technically well done and you discover something really interesting: accorgint to your map there is a census tract where everyone is male! that is very very weird and should be investigated! It would have beem great if you looked into it and found why :-)

In [ ]:
# Male & Female (Age between 25 and 29)
fig = plt.figure(figsize=(15,10))
ax = fig.add_subplot(121)
maleDE18_2529.plot(column="male2529_fraction", #scheme='Quantiles',
            ax=ax, legend=True, vmin=0.0, vmax=0.15)
ax.axis('off');
plt.title("Male fraction 2018 (Age 25-29)")

ax = fig.add_subplot(122)
femaleDE18_2529.plot(column="female2529_fraction", 
               ax=ax, legend=True, vmin=0.0, vmax=0.15);
ax.axis('off');
plt.title("Female fraction 2018 (Age 25-29)");